# 1. Importing Packages

In [3]:
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL,NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np

In [13]:
def create_eigs(matrix,num_ancillae, num_time_slices, negative_evals):
    ne_qfts=[None,None]
    if negative_evals:
        num_ancillae+=1 
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]
    
    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

def fidelity(hhl,ref):
    solution_hhl_normed=hhl/np.linalg.norm(hhl)
    solution_ref_normed=ref/np.linalg.norm(ref)
    
    fidelity =state_fidelity(solution_hhl_normed,solution_ref_normed)
    print("Fidelity: %f"%fidelity)

In [14]:
matrix = [[1, -1/3], [-1/3, 1]]
vector = [1, 0]

orig_size=len(vector)
matrix,vector,trunc_powerdim,trunc_hermitian = HHL.matrix_resize(matrix,vector)

eigs=create_eigs(matrix,3,50,False)
num_q,num_a=eigs.get_register_sizes()

init_state=Custom(num_q,state_vector=vector)

reci=LookupRotation(negative_evals=eigs._negative_evals,evo_time=eigs._evo_time)

algo=HHL(matrix, vector, trunc_powerdim, trunc_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

In [15]:
result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:\t\t", np.round(result['solution'], 5))

result_ref = NumPyLSsolver(matrix, vector).run()
print("Classical Solution:\t", np.round(result_ref['solution'], 5))

print("Probability:\t\t %f" % result['probability_result'])
fidelity(result['solution'], result_ref['solution'])

Solution:		 [1.13586-0.j 0.40896+0.j]
Classical Solution:	 [1.125 0.375]
Probability:		 0.056291
Fidelity: 0.999432
